# StyleGAN2

Notebooks:
- this notebook: https://github.com/woctezuma/steam-stylegan2
- another of my notebooks: https://github.com/woctezuma/steam-stylegan

References:
- the original StyleGAN2 repository: https://github.com/NVlabs/stylegan2
- my fork of StyleGAN2: https://github.com/woctezuma/stylegan2



## Request more RAM (once per session)

Crash the Google Colab session to have access to more memory.

Reference: https://github.com/googlecolab/colabtools/issues/253#issuecomment-551056637

In [0]:
d = [ '0' ]
while(True):
  d += d

## Installing StyleGAN2

### The official StyleGAN2 implementation

In [1]:
%cd /content/

/content


Clone my fork:

In [2]:
!rm -rf stylegan2/
!git clone https://github.com/woctezuma/stylegan2.git

Cloning into 'stylegan2'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 119 (delta 12), reused 15 (delta 6), pack-reused 98
Receiving objects: 100% (119/119), 1.02 MiB | 1.38 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [3]:
pushd

['/content']

In [4]:
%cd stylegan2/

/content/stylegan2


Switch to my branch for Google Colab:

In [0]:
!git checkout google-colab-fixed

In [6]:
!nvcc test_nvcc.cu -o test_nvcc -run

CPU says hello.
GPU says hello.


In [7]:
popd

/content
popd -> /content


### (optional) An alternative implementation in PyTorch

In [0]:
pip install stylegan2_pytorch

## Mounting Google Drive

In [8]:
!pip install Google-Colab-Transfer

In [9]:
import colab_transfer

colab_path = colab_transfer.get_path_to_home_of_local_machine()
drive_path = colab_transfer.get_path_to_home_of_google_drive()

print('Disk of the virtual matchine: {}'.format(colab_path))
print('Google Drive: {}'.format(drive_path))

Disk of the virtual matchine: /content/
Google Drive: /content/drive/My Drive/


In [10]:
colab_transfer.mount_google_drive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Data parameters

In [0]:
folder_name = 'datasets/steam/'

## Preparing datasets (once)

### Importing data from Google Drive

Data consists of 14,035 vertical Steam banners, resized from 300x450 to 256x256 resolution.

In [0]:
colab_transfer.copy_file(file_name='256x256.zip')

File /content/256x256.zip already exists. Copy skipped.


In [0]:
!unzip 256x256.zip -d /content/

### Preparing data for StyleGAN2

In [0]:
!python stylegan2/dataset_tool.py create_from_images '/content/datasets/steam' '/content/256x256'

Loading images from "/content/256x256"
Creating dataset "/content/datasets/steam"
Added 14035 images.


### Archive prepared datasets to Google Drive

In [0]:
colab_transfer.copy_folder(folder_name,
                           source=colab_path,
                           destination=drive_path)

Copying /content/datasets/steam/ to /content/drive/My Drive/datasets/steam/


## Importing prepared datasets from Google Drive

In [12]:
colab_transfer.copy_folder(folder_name,
                           source=drive_path,
                           destination=colab_path)

Copying /content/drive/My Drive/datasets/steam/ to /content/datasets/steam/


## Training networks

There is no need to edit `training/training_loop.py`, thanks to automatic resuming from the latest snapshot, implemented in my fork.

Otherwise, one would have to manually edit the file from within the Google Colab session!

### Train with the official StyleGAN2 implementation

Our Steam data consists of ~14k images, which exhibits a similar dataset size to the [FFHQ dataset](https://github.com/NVlabs/ffhq-dataset) (70k images, so 5 times larger).

Therefore, the parameters used for our data are inspired from the ones described in the StyleGAN2 README for the FFHQ dataset:
- `--mirror-augment=true`: data augmentation with horitontal mirroring,
- `--total-kimg=5000`: during training with our Steam data, StyleGAN2 will be shown 5 times fewer images than during training with the FFHQ data (the default value used for FFHQ is 25 million images: `--total-kimg=25000`).

Model snapshots are directly saved to Google Drive (`--result-dir='/content/drive/My Drive/results'`).

In [0]:
!python stylegan2/run_training.py --config=config-e --metrics=none \
   --data-dir='/content/datasets' --dataset=steam \
   --mirror-augment=true \
   --total-kimg=5000 \
   --result-dir='/content/drive/My Drive/results' \


### (optional) Train with an alternative implementation in PyTorch

In [0]:
# !stylegan2_pytorch --data /content/256x256 --image-size 256 \
#    --results_dir '/content/drive/My Drive/pytorch/results' \
#    --models_dir '/content/drive/My Drive/pytorch/models' \
#    --name steam   

## Generating images

### Load the snapshots from Google Drive

In [0]:
result_root_folder = 'results/'

In [0]:
result_folder_name_to_load = '00000-stylegan2-steam-1gpu-config-e'

In [0]:
colab_transfer.copy_folder(result_folder_name_to_load,
                           source=drive_path + result_root_folder,
                           destination=colab_path + result_root_folder)

### Using the first snapshot

In [0]:
# Generate 1000 random images without truncation
!python stylegan2/run_generator.py generate-images \
  --seeds=0-999 --truncation-psi=1.0 \
  --network=results/00000-stylegan2-steam-1gpu-config-e/network-snapshot-000000.pkl

### Using the last snapshot

In [0]:
# Generate 1000 random images without truncation
!python stylegan2/run_generator.py generate-images \
  --seeds=0-999 --truncation-psi=1.0 \
  --network=results/00000-stylegan2-steam-1gpu-config-e/networks-final.pkl